<a href="https://colab.research.google.com/github/riccardocappi/Text-Adversarial-Attack/blob/adversarial-training/NLP_TextAttack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing
# sex

In [1]:
!pip install textattack

^C
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
# @title
import textattack
import transformers
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.transformations import WordSwapEmbedding
from textattack.search_methods import GreedyWordSwapWIR

ModuleNotFoundError: No module named 'textattack'

### BAE: BERT-based Adversarial Examples for Text Classification

In [ ]:
# Load model, tokenizer, and model_wrapper
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
from textattack.datasets import HuggingFaceDataset
from textattack import Attacker
from textattack.attack_recipes import GeneticAlgorithmAlzantot2018

In [ ]:
# Load dataset
dataset = HuggingFaceDataset("imdb", None, "test", shuffle=True)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

textattack: Loading datasets dataset imdb, split test.


# Attack #

In [ ]:
attack = GeneticAlgorithmAlzantot2018.build(model_wrapper)

attack_args = textattack.AttackArgs(num_examples=10, parallel=True)
attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:51<00:00, 9.35MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp4duacm__.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.
textattack: Downloading https://textattack.s3.amazonaws.com/constraints/semantics/language-models/alzantot-goog-lm.
100%|██████████| 3.87G/3.87G [05:19<00:00, 12.1MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpxl2rfowl.zip to /root/.cache/textattack/constraints/semantics/language-models/alzantot-goog-lm.
textattack: Successfully saved constraints/semantics/language-models/alzantot-goog-lm to cache.
Recovering graph.
INFO:tensorflow:Recovering Graph /root/.cache/textattack/constraints/semantics/language-models/alzantot-goog-lm/graph-2016-09-10-gpu.pbtxt
Recovering checkpoint /root/.cache/textattack/constraints/semantics/language-models/alzantot-goog-l

TypeError: cannot pickle 'Graph' object

### Try with yelp dataset

In [ ]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

In [ ]:
# Load dataset
dataset = HuggingFaceDataset("yelp_polarity", None, "test", shuffle=True)

In [ ]:
attack = GeneticAlgorithmAlzantot2018.build(model_wrapper)
#sesso

attack_args = textattack.AttackArgs(num_examples=10, parallel=True)
attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

# ADVERSARIAL TRAINING & DATA AUGMENTATION #


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [1]:
BERTModel = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")

NameError: name 'transformers' is not defined

In [ ]:
BERTModel.num_parameters()

109483778

In [ ]:
BERTModel

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

# FineTune function #

In [ ]:
def fineTuneModel(model, train_dataset,val_dataset, epochs=20, batch_s=16):



  for param in model.parameters():
    param.require_grad = False

  train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_s, shuffle=True)
  val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_s)

  optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
  loss_fn = torch.nn.CrossEntropyLoss()

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  #Training
  for epoch in range(epochs):
      model.train()

      for batch in train_dataloader:

          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          labels = batch['labels'].to(device)

          # Forward pass
          outputs = model(input_ids, attention_mask=attention_mask)
          logits = outputs.logits

          # Loss calculation and backpropagation
          loss = loss_fn(logits, labels)
          loss.backward()

          optimizer.step()
          optimizer.zero_grad()
